# Black Hole Spectra

Black hole spectra can be generated by combining a `BlackHoles` (for particle, `BlackHole` for parametric) object with an ``EmissionModel``, translating the physical properties of the blackhole(s) (e.g. `mass`, `accretion_rate`, etc.) to a spectral energy distribution.

These models are described in detail in the [emission model docs](../../emission_models/emission_models.rst). Here, we'll use an instance of a ``UnifiedAGN`` model for demonstration purposes.

The following sections demonstrate the generation of combined spectra (which is the same for both parametric and particle ``BlackHoles``) and per-particle spectra. 

In [ ]:
import numpy as np
from unyt import K, Mpc, Msun, cm, deg, yr

from synthesizer import Grid
from synthesizer.emission_models import (
    Greybody,
    UnifiedAGN,
)
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.parametric import BlackHole

# Get the NLR and BLR grids
nlr_grid = Grid("test_grid_agn-nlr")
blr_grid = Grid("test_grid_agn-blr")


# Initialise the BlackHole object setting most of the key attributes
blackhole = BlackHole(
    mass=1e8 * Msun,
    inclination=60 * deg,
    accretion_rate_eddington=0.1,
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.1,
    metallicity=0.01,
    theta_torus=20 * deg,
)

# Initialise the UnifiedAGN model
uniagn = UnifiedAGN(
    nlr_grid,
    blr_grid,
    ionisation_parameter_nlr=0.01,
    hydrogen_density_nlr=1e4 * cm**-3,
    ionisation_parameter_blr=0.1,
    hydrogen_density_blr=1e10 * cm**-3,
    torus_emission_model=Greybody(1000 * K, 1.5),
)


## Integrated spectra

To generate integrated spectra we simply call the component's ``get_spectra`` method. This method will populate the component's ``spectra`` attribute with a dictionary containing [Sed objects](../emission_objects/sed_example.ipynb) for each spectra in the ``EmissionModel``.
It will also return the spectra at the root of the ``EmissionModel``.

In [ ]:
# Get the spectra using a unified agn model (instantiated elsewhere)
spectra = blackhole.get_spectra(uniagn)

fig, ax = blackhole.plot_spectra(
    show=True,
    ylimits=(10**27.5, 10**34.0),
    figsize=(10, 8),
)

print(blackhole.model_param_cache)

### Including dust attenuation

We can also generate spectra including attenuation and emission from diffuse dust along the line of sight to the black hole. This is now possible directly with the `UnifiedAGN` by passing a dust curve. The optical depth (`tau_v`) must be available on the emitter (the blackhole) or set by the emission model.  

In [ ]:
tau_v = 0.5

# Initialise the UnifiedAGN model
uniagn_attenuated = UnifiedAGN(
    nlr_grid,
    blr_grid,
    ionisation_parameter_nlr=0.01,
    hydrogen_density_nlr=1e4 * cm**-3,
    ionisation_parameter_blr=0.1,
    hydrogen_density_blr=1e10 * cm**-3,
    torus_emission_model=Greybody(1000 * K, 1.5),
    diffuse_dust_curve=PowerLaw(slope=-1.0),
    tau_v=tau_v,
)

We then follow the same process of calling ``get_spectra`` with the new model.
The plot here shows luminosity rather than spectral energy density.

In [ ]:
spectra = blackhole.get_spectra(uniagn_attenuated)

fig, ax = blackhole.plot_spectra(
    quantity_to_plot="luminosity",
    figsize=(6, 4),
    spectra_to_plot=["intrinsic", "attenuated"],
)

The spectra returned by ``get_spectra`` is the "dust_emission" spectra at the root of the emission model.

In [ ]:
print(spectra)

However, all the spectra are stored within a dictionary under the ``spectra`` attribute.

In [ ]:
print(blackhole.spectra)

## Particle spectra

To demonstrate the particle spectra functionality we first generate some mock particle black hole data, and initialise a ``BlackHoles`` object.

In [ ]:
from synthesizer.particle import BlackHoles

# Make fake properties
n = 4
masses = 10 ** np.random.uniform(low=7, high=9, size=n) * Msun
coordinates = np.random.normal(0, 1.5, (n, 3)) * Mpc
accretion_rates = 10 ** np.random.uniform(low=-2, high=1, size=n) * Msun / yr
metallicities = np.full(n, 0.01)

# And get the black holes object
blackholes = BlackHoles(
    masses=masses,
    coordinates=coordinates,
    accretion_rates=accretion_rates,
    metallicities=metallicities,
    ionisation_parameter_nlr=0.01,
    hydrogen_density_nlr=1e4 * cm**-3,
    ionisation_parameter_blr=0.1,
    hydrogen_density_blr=1e10 * cm**-3,
)

To generate a spectra for each black hole (per particle) we use the same emission model, but we need to tell the model to produce a spectrum for each particle. This is done by setting the ``per_particle`` flag to ``True`` on the model.

In [ ]:
uniagn_attenuated.set_per_particle(True)

With that done we just call the same ``get_spectra`` method on the component, and the particle spectra will be stored in the ``particle_spectra`` attribute of the component.

In [ ]:
spectra = blackholes.get_spectra(uniagn_attenuated, verbose=True)

Again, the returned spectra is the "dust_emission" spectra from the root of the model.

In [ ]:
print(spectra)

While the spectra produced by ``get_particle_spectra`` are stored in a dictionary under the ``particle_spectra`` attribute.

In [ ]:
print(blackholes.particle_spectra)

### Integrating spectra

The integrated spectra are automatically produced alongside per particle spectra. However, if we wanted to explictly get the integrated spectra from the particle spectra we just generated (for instance if we had made some modification after generation), we can call the ``integrate_particle_spectra`` method. This method will sum the individual spectra and populate the ``spectra`` dictionary.

Note, we can also integrate individual spectra using the ``Sed.sum()`` method.

In [ ]:
print(blackholes.spectra)
blackholes.integrate_particle_spectra()
print(blackholes.spectra)

fig, ax = blackholes.plot_spectra(
    show=True, ylimits=(10**28.5, 10**34.0), figsize=(9, 7.2)
)

## Printing Used Parameters

During spectra generation, emission models cache the parameters they extract and use from the emitter. These cached parameters can be printed in a nicely formatted table to inspect which values were actually used by each model.

In [ ]:
# Print the cached parameters used by the models
blackholes.print_used_parameters()